In [ ]:
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from data_utils import preprocessDataset
import numpy as np
from scipy.stats import multivariate_normal
import copy
from aif360.metrics import ClassificationMetric
import math
plt.rcParams['axes.ymargin'] = 0.5
plt.rcParams['axes.axisbelow'] = True

In [ ]:
def getBayesOptimalResults(dim=100):
    with open('/media/data_dump/anonymous/aaai_2023_data/synthetic/means.pkl', 'rb') as f:
        means = pickle.load(f)
    _, test_dataset_original = preprocessDataset('/media/data_dump/anonymous/aaai_2023_data/synthetic/raw/original_train.csv', '/media/data_dump/anonymous/aaai_2023_data/synthetic/raw/original_test.csv', 'synthetic')
    x = test_dataset_original.features[:,:-1]
    z = test_dataset_original.protected_attributes.squeeze()
    y = []
    var = 1
    #group_0_0_pdf = np.random.multivariate_normal(mean=means[0], cov=var*np.eye(1))
    #group_0_1_pdf = np.random.multivariate_normal(mean=means[1], cov=var*np.eye(1))
    #group_1_0_pdf = np.random.multivariate_normal(mean=means[2], cov=var*np.eye(1))
    #group_1_1_pdf = np.random.multivariate_normal(mean=means[3], cov=var*np.eye(1))
    for i in range(len(x)):
        if z[i] == 0:
            # first index y, second z
            pdf_0_0 = multivariate_normal.pdf(x[i], mean=means[0], cov=var*np.eye(dim))
            pdf_1_0 = multivariate_normal.pdf(x[i], mean=means[2], cov=var*np.eye(dim))
            if pdf_0_0 > pdf_1_0:
                y.append(0)
            else:
                y.append(1)
        elif z[i] == 1:
            # first index y, second z
            pdf_0_1 = multivariate_normal.pdf(x[i], mean=means[1], cov=var*np.eye(dim))
            pdf_1_1 = multivariate_normal.pdf(x[i], mean=means[3], cov=var*np.eye(dim))
            if pdf_0_1 > pdf_1_1:
                y.append(0)
            else:
                y.append(1)
#     for i in range(len(x)):
#         pdfs = [multivariate_normal.pdf(x[i], mean=means[0], cov=var*np.eye(dim)),
#         multivariate_normal.pdf(x[i], mean=means[1], cov=var*np.eye(dim)),
#         multivariate_normal.pdf(x[i], mean=means[2], cov=var*np.eye(dim)),
#         multivariate_normal.pdf(x[i], mean=means[3], cov=var*np.eye(dim))]
#         correctIndex = np.argmax(pdfs)
#         if correctIndex == 0 or correctIndex == 1:
#             y.append(0)
#         elif correctIndex == 2 or correctIndex == 3:
#             y.append(1)
    pred_test_set = copy.deepcopy(test_dataset_original)
    pred_test_set.labels = np.asarray(y)
    privileged_groups = [{'sensitive':1}]
    unprivileged_groups = [{'sensitive': 0}]
    return ClassificationMetric(test_dataset_original, pred_test_set, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

In [ ]:
def max_mean_plots(files, metric='eod', testSet='orig', dataset='synthetic', statType='mean', plotLimits=None, lessBiasMode=False):
    if dataset == 'compas':
        omitCls = ['meta_fair_fdr']
    else:
        omitCls = []
    axisTitles = {
        'dp': 'Statistical Parity Diff.',
        'eod': 'Equal Opportunity Difference'
    }
    markers = {
        'lr': '+',
        'jiang_nachum': 'o',
        'rew':'o',
        'fairgan':'o',
        'exp_grad_eodds':'^',
        'meta_fair_fdr':'^',
        'prej_remover':'^',
        'gerry_fair': '^',
        'cal_eq':'s',
        'eq':'s',
        'reject':'s'
    }
    colors = {
        'lr': '#1f77b4',
        'jiang_nachum': '#ff7f0e',
        'rew':'#2ca02c',
        'fairgan':'#d62728',
        'exp_grad_eodds':'#9467bd',
        'meta_fair_fdr':'#8c564b',
        'prej_remover':'#e377c2',
        'gerry_fair': '#7f7f7f',
        'cal_eq':'#bcbd22',
        'eq':'#17becf',
        'reject':'#1f77b4'
    }
    fig, ax = plt.subplots(ncols=4)
    fig.subplots_adjust(wspace=0.4)
    fig.set_figheight(3)
    fig.set_figwidth(12)
    overallStats = {}
    fixed_bp_variable_bn_stats = {}
    fixed_bn_variable_bp_stats = {}
    for file in files:
        if file == 'base':
            file = 'lr'
        if file not in omitCls:
            overallStats[file] = [[],[]]
            fixed_bp_variable_bn_stats[file] = [[],[]]
            fixed_bn_variable_bp_stats[file] = [[],[]]
    if lessBiasMode == True:
        start_range = 5
    else:
        start_range = 1
    end_range = 11
    for bp,beta_pos in enumerate(range(start_range,11)):
        beta_pos /= 10
        for bn,beta_neg in enumerate(range(start_range,11)):
            beta_neg /= 10
            # Calculating required Quantities
            for file in files:
                if file in omitCls:
                    continue
                results = files[file]
                try:
                    if metric == 'dp':
                        y_bal = abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['balanced'].statistical_parity_difference())
                        y_biased = abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['biased'].statistical_parity_difference())
                        y_orig = abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['original'].statistical_parity_difference())
                    elif metric == 'eod':
                        y_bal = abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['balanced'].equal_opportunity_difference())
                        y_biased = abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['biased'].equal_opportunity_difference())
                        y_orig = abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['original'].equal_opportunity_difference())
                    # Calculate accuracy
                    x_err_rate_bal = (1 - abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['balanced'].accuracy()))
                    x_err_rate_biased = (1 - abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['biased'].accuracy()))
                    x_err_rate_orig = (1 - abs(results['undersample'][f'imbalance_{str(beta_pos)}_{str(beta_neg)}']['original'].accuracy()))
                except TypeError as te:
                    continue
                if testSet == 'bal':
                    y = y_bal
                    x = x_err_rate_bal
                elif testSet == 'biased':
                    y = y_biased
                    x = x_err_rate_biased
                elif testSet == 'orig':
                    y = y_orig
                    x = x_err_rate_orig
                # Renaming 'base' to 'lr'
                if file == 'base':
                    file = 'lr'
                    overallStats[file][0].append(x)
                    overallStats[file][1].append(y)
                    if beta_pos == 1.0:
                        fixed_bp_variable_bn_stats[file][0].append(x)
                        fixed_bp_variable_bn_stats[file][1].append(y)
                    if beta_neg == 1.0:
                        fixed_bn_variable_bp_stats[file][0].append(x)
                        fixed_bn_variable_bp_stats[file][1].append(y)
                else:
                    overallStats[file][0].append(x)
                    overallStats[file][1].append(y)
                    if beta_pos == 1.0:
                        fixed_bp_variable_bn_stats[file][0].append(x)
                        fixed_bp_variable_bn_stats[file][1].append(y)
                    if beta_neg == 1.0:
                        fixed_bn_variable_bp_stats[file][0].append(x)
                        fixed_bn_variable_bp_stats[file][1].append(y)
    ##############
    # Overall Mean
    ##############
    del overallStats['lr']
    for i in overallStats:
        if statType == 'max':
            ax[1].errorbar(x=max(overallStats[i][0]), y=max(overallStats[i][1]), label=i, fmt=markers[i], color=colors[i])
        else:
            if np.mean(overallStats[i][0]) > plotLimits[dataset][0][1] or np.mean(overallStats[i][1]) > plotLimits[dataset][1][1]:
                ax[1].errorbar(x=np.mean(overallStats[i][0]), y=np.mean(overallStats[i][1]), label=i, fmt=markers[i], color=colors[i])
            else:
                temp = ax[1].errorbar(x=np.mean(overallStats[i][0]), y=np.mean(overallStats[i][1]), xerr=np.std(overallStats[i][0]), yerr=np.std(overallStats[i][1]), label=i, fmt=markers[i], color=colors[i])
                [bar.set_alpha(0.3) for bar in temp[2]]
                [cap.set_alpha(0.3) for cap in temp[1]]
        ax[1].set_ylim(plotLimits[dataset][1])
        ax[1].set_xlim(plotLimits[dataset][0])
    # D2 distribution lines
    #if dataset != 'synthetic':
    if metric == 'eod':
        ax[1].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[1].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].equal_opportunity_difference()))
    elif metric == 'dp':
        ax[1].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[1].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].statistical_parity_difference()))
    ax[1].set_xlabel('Error Rate')
    #ax[1].set_ylabel(axisTitles[metric])
    ax[1].set_title('Mean Over all settings', fontsize=10)
#     else:
#         bayes_optimal_results = getBayesOptimalResults()
#         if metric == 'eod':
#             ax[1].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[1].axhline(y=abs(bayes_optimal_results.equal_opportunity_difference()))
#         elif metric == 'dp':
#             ax[1].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[1].axhline(y=abs(bayes_optimal_results.statistical_parity_difference()))
#         ax[1].set_xlabel('Error Rate')
#         ax[1].set_ylabel(axisTitles[metric])
#         ax[1].set_title('Mean Over all settings', fontsize=10)

    #######################
    # BP fixed, BN variable
    #######################
    del fixed_bp_variable_bn_stats['lr']
    for i in fixed_bp_variable_bn_stats:
        if statType == 'max':
            ax[2].errorbar(x=max(fixed_bp_variable_bn_stats[i][0]), y=max(fixed_bp_variable_bn_stats[i][1]), label=i, fmt=markers[i], color=colors[i])
        else:
            if np.mean(fixed_bp_variable_bn_stats[i][0]) > plotLimits[dataset][0][1] or np.mean(fixed_bp_variable_bn_stats[i][1]) > plotLimits[dataset][1][1]:
                ax[2].errorbar(x=np.mean(fixed_bp_variable_bn_stats[i][0]), y=np.mean(fixed_bp_variable_bn_stats[i][1]), label=i, fmt=markers[i], color=colors[i])
            else:
                temp = ax[2].errorbar(x=np.mean(fixed_bp_variable_bn_stats[i][0]), xerr=np.std(fixed_bp_variable_bn_stats[i][0]), y=np.mean(fixed_bp_variable_bn_stats[i][1]), yerr=np.std(fixed_bp_variable_bn_stats[i][1]), label=i, fmt=markers[i], color=colors[i])
                [bar.set_alpha(0.3) for bar in temp[2]]
                [cap.set_alpha(0.3) for cap in temp[1]]
        ax[2].set_ylim(plotLimits[dataset][1])
        ax[2].set_xlim(plotLimits[dataset][0])
    # D2 distribution lines
    #if dataset != 'synthetic':
    if metric == 'eod':
        ax[2].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[2].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].equal_opportunity_difference()))
    elif metric == 'dp':
        ax[2].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[2].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].statistical_parity_difference()))
    ax[2].set_xlabel('Error Rate')
    #ax[2].set_ylabel(axisTitles[metric])
    ax[2].set_title(r'Mean over all $β_{pos}$=1.0', fontsize=10)
    #else:
#     bayes_optimal_results = getBayesOptimalResults()
#     if metric == 'eod':
#         ax[2].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#         ax[2].axhline(y=abs(bayes_optimal_results.equal_opportunity_difference()))
#     elif metric == 'dp':
#         ax[2].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#         ax[2].axhline(y=abs(bayes_optimal_results.statistical_parity_difference()))
#     ax[2].set_xlabel('Error Rate')
#     ax[2].set_ylabel(axisTitles[metric])
#     ax[2].set_title(r'Mean over all $β_{pos}$=1.0', fontsize=10)
    
    #######################
    # BN fixed, BP variable
    #######################
    del fixed_bn_variable_bp_stats['lr']
    for i in fixed_bn_variable_bp_stats:
        if statType == 'max':
            ax[3].errorbar(x=max(fixed_bn_variable_bp_stats[i][0]), y=max(fixed_bn_variable_bp_stats[i][1]), label=i, fmt=markers[i], color=colors[i])
        else:
            if np.mean(fixed_bn_variable_bp_stats[i][0]) > plotLimits[dataset][0][1] or np.mean(fixed_bn_variable_bp_stats[i][1]) > plotLimits[dataset][1][1]:
                ax[3].errorbar(x=np.mean(fixed_bn_variable_bp_stats[i][0]), y=np.mean(fixed_bn_variable_bp_stats[i][1]), label=i, fmt=markers[i], color=colors[i])
            else:
                temp = ax[3].errorbar(x=np.mean(fixed_bn_variable_bp_stats[i][0]), xerr=np.std(fixed_bn_variable_bp_stats[i][0]), y=np.mean(fixed_bn_variable_bp_stats[i][1]), yerr=np.mean(fixed_bn_variable_bp_stats[i][1]), label=i, fmt=markers[i], color=colors[i])
                [bar.set_alpha(0.3) for bar in temp[2]]
                [cap.set_alpha(0.3) for cap in temp[1]]
        ax[3].set_ylim(plotLimits[dataset][1])
        ax[3].set_xlim(plotLimits[dataset][0])
    # D2 distribution lines
    #if dataset != 'synthetic':
    if metric == 'eod':
        ax[3].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[3].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].equal_opportunity_difference()))
    elif metric == 'dp':
        ax[3].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[3].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].statistical_parity_difference()))
    ax[3].set_xlabel('Error Rate')
    #ax[3].set_ylabel(axisTitles[metric])
    ax[3].set_title(r'Mean over all $β_{neg}$=1.0', fontsize=10)
#     else:
#         bayes_optimal_results = getBayesOptimalResults()
#         if metric == 'eod':
#             ax[3].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[3].axhline(y=abs(bayes_optimal_results.equal_opportunity_difference()))
#         elif metric == 'dp':
#             ax[3].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[3].axhline(y=abs(bayes_optimal_results.statistical_parity_difference()))
#         ax[3].set_xlabel('Error Rate')
#         ax[3].set_ylabel(axisTitles[metric])
#         ax[3].set_title(r'Mean over all $β_{neg}$=1.0', fontsize=10)
    
    ################
    # Frequency Plot
    ################
    cls_freqs = {}
    for i in files:
        cls_freqs[i] = 0
    del cls_freqs['base']
    for beta_pos in range(start_range,11):
        beta_pos /= 10
        for beta_neg in range(start_range,11):
            beta_neg /=10
            current_coord = {}
            for file in files:
                try:
                    if metric == 'eod':
                        current_coord[file] = (1 - files[file]['undersample'][f'imbalance_{beta_pos}_{beta_neg}']['original'].accuracy(), abs(files[file]['undersample'][f'imbalance_{beta_pos}_{beta_neg}']['original'].equal_opportunity_difference()))
                    elif metric == 'dp':
                        current_coord[file] = (1 - files[file]['undersample'][f'imbalance_{beta_pos}_{beta_neg}']['original'].accuracy(), abs(files[file]['undersample'][f'imbalance_{beta_pos}_{beta_neg}']['original'].statistical_parity_difference()))
                except TypeError as te:
                    continue
            del current_coord['base']
            sorted_acc = sorted(list(current_coord.items()), key=lambda x: x[1][0])
            sorted_fairness = sorted(list(current_coord.items()), key=lambda x: x[1][1])
            k = 5
            top_k_classifiers = list(set([i[0] for i in sorted_acc][:k]).intersection(set([i[0] for i in sorted_fairness][:k])))
            for i in current_coord:
                if current_coord[i][0] < 0.5 and current_coord[i][1] < 0.5:
                    flag = 0
                    for j in current_coord:
                        if j != i:
                            # Check whether a point exists
                            if current_coord[j][0] < current_coord[i][0] and current_coord[j][1] < current_coord[i][1]:
                                flag = 1
                                break
                    if flag == 0 and i in top_k_classifiers:
                        cls_freqs[i] += 1
    
    ###############################
    # Original Split Results
    ###############################
    for i in files:
        if i not in omitCls:
            x = 1 - files[i]['undersample'][f'imbalance_1.0_1.0']['original'].accuracy()
            if metric == 'eod':
                y=abs(files[i]['undersample'][f'imbalance_1.0_1.0']['original'].equal_opportunity_difference())
            elif metric == 'dp':
                y=abs(files[i]['undersample'][f'imbalance_1.0_1.0']['original'].statistical_parity_difference())
            if i == 'base':
                i = 'lr'
                continue
            ax[0].errorbar(x, y, label=i, fmt=markers[i], color=colors[i])
    # D2 distribution lines
    #if dataset != 'synthetic':
    if metric == 'eod':
        ax[0].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[0].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].equal_opportunity_difference()))
    elif metric == 'dp':
        ax[0].axvline(x=(1.0 - files['base']['undersample']['imbalance_1.0_1.0']['original'].accuracy()))
        ax[0].axhline(y=abs(files['base']['undersample']['imbalance_1.0_1.0']['original'].statistical_parity_difference()))
#     else:
#         bayes_optimal_results = getBayesOptimalResults()
#         if metric == 'eod':
#             ax[0].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[0].axhline(y=abs(bayes_optimal_results.equal_opportunity_difference()))
#         elif metric == 'dp':
#             ax[0].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[0].axhline(y=abs(bayes_optimal_results.statistical_parity_difference()))
    ax[0].set_xlabel('Error Rate')
    ax[0].set_ylabel(axisTitles[metric])
    ax[0].set_title('Results on Original Split', fontsize=10)
    ax[0].set_ylim(plotLimits[dataset][1])
    ax[0].set_xlim(plotLimits[dataset][0])
    
    if dataset == 'credit':
        plt.legend(bbox_to_anchor=(1.1,1.1))
    elif dataset == 'adult':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    elif dataset == 'compas':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    elif dataset == 'bank':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    elif dataset == 'synthetic':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    plt.show()
    
    if lessBiasMode == True:
        fig.savefig(f'{dataset}_{metric}_lessBias_underRepresentation.png', dpi=300, bbox_inches='tight')
    else:
        fig.savefig(f'{dataset}_{metric}_underRepresentation.png', dpi=300, bbox_inches='tight')
    
    # Dump Pareto Plot data
    with open(f'pareto_{dataset}_{metric}.pkl', 'wb') as f:
        pickle.dump(cls_freqs, f)
    plt.figure()
    plt.bar(range(len(cls_freqs)), cls_freqs.values(), tick_label=list(cls_freqs.keys()))
    plt.xticks(rotation=90, fontsize=8)
    plt.title('Pareto Optimality Frequency', fontsize=9)
    plt.show()

In [ ]:
import gc
try:
    del files
    gc.collect()
except:
    pass

dataset='credit'
if dataset == 'bank':
    files={
        'base': f'/media/data_dump/anonymous/aaai_2023_data/results/base__eop__{dataset}__lr.pkl',
        'jiang_nachum': f'/media/data_dump/anonymous/aaai_2023_data/results/jiang_nachum__eop__{dataset}__lr.pkl',
        'rew':f'/media/data_dump/anonymous/aaai_2023_data/results/rew__eop__{dataset}__lr.pkl',
        'exp_grad_eodds':f'/media/data_dump/anonymous/aaai_2023_data/results/exp_grad__eodds__{dataset}__lr.pkl',
        'meta_fair_fdr':f'/media/data_dump/anonymous/aaai_2023_data/results/meta_fair__fdr__{dataset}__lr.pkl',
        'prej_remover':f'/media/data_dump/anonymous/aaai_2023_data/results/prej_remover__eop__{dataset}__lr.pkl',
        'gerry_fair': f'/media/data_dump/anonymous/aaai_2023_data/results/gerry_fair__eop__{dataset}__lr.pkl',
        'cal_eq':f'/media/data_dump/anonymous/aaai_2023_data/results/cal_eq__eop__{dataset}__lr.pkl',
        'eq':f'/media/data_dump/anonymous/aaai_2023_data/results/eq__eop__{dataset}__lr.pkl',
        'reject':f'/media/data_dump/anonymous/aaai_2023_data/results/reject__eop__{dataset}__lr.pkl',
    }
else:
    files={
        'base': f'/media/data_dump/anonymous/aaai_2023_data/results/base__eop__{dataset}__lr.pkl',
        'jiang_nachum': f'/media/data_dump/anonymous/aaai_2023_data/results/jiang_nachum__eop__{dataset}__lr.pkl',
        'rew':f'/media/data_dump/anonymous/aaai_2023_data/results/rew__eop__{dataset}__lr.pkl',
        'fairgan':f'/media/data_dump/anonymous/aaai_2023_data/results/fairgan__eop__{dataset}__lr.pkl',
        'exp_grad_eodds':f'/media/data_dump/anonymous/aaai_2023_data/results/exp_grad__eodds__{dataset}__lr.pkl',
        'meta_fair_fdr':f'/media/data_dump/anonymous/aaai_2023_data/results/meta_fair__fdr__{dataset}__lr.pkl',
        'prej_remover':f'/media/data_dump/anonymous/aaai_2023_data/results/prej_remover__eop__{dataset}__lr.pkl',
        'gerry_fair': f'/media/data_dump/anonymous/aaai_2023_data/results/gerry_fair__eop__{dataset}__lr.pkl',
        'cal_eq':f'/media/data_dump/anonymous/aaai_2023_data/results/cal_eq__eop__{dataset}__lr.pkl',
        'eq':f'/media/data_dump/anonymous/aaai_2023_data/results/eq__eop__{dataset}__lr.pkl',
        'reject':f'/media/data_dump/anonymous/aaai_2023_data/results/reject__eop__{dataset}__lr.pkl',
    }
for i in files:
    with open(files[i], 'rb') as f:
        files[i] = pickle.load(f)

In [ ]:
metric='eod'
if metric == 'eod':
    plotLimits = {
            'synthetic': ([-0.05, 0.3], [-0.03,0.2]),
            'adult': ([0, 0.4], [-0.01, 0.25]),
            'bank': ([0, 0.5], [-0.01, 0.35]),
            'compas': ([0.05, 0.5], [-0.01, 0.2]),
            'credit': ([0, 0.4], [-0.01, 0.2])
        }
else:
    plotLimits = {
            'synthetic': ([-0.1, 0.55], [0, 0.4]),
            'adult': ([-0.1, 0.38], [-0.01, 0.4]),
            'bank': ([0, 0.6], [-0.01, 0.4]),
            'compas': ([0.05, 0.5], [-0.01, 0.3]),
            'credit': ([0, 0.4], [-0.01, 0.2])
        }
max_mean_plots(files, metric=metric, dataset=dataset, plotLimits=plotLimits, lessBiasMode=True)
#max_mean_plots(files, metric=metric, dataset=dataset, plotLimits=plotLimits)

In [ ]:
def plotParetoBars(mode='', metric='', lessBiasMode=False):
    plt.figure(figsize=(20,3))
    perClassifierResults = {}
    if mode == 'labelBias':
        with open(f'pareto_labelBias_adult_{metric}.pkl', 'rb') as f:
            cls_freqs = pickle.load(f)
    else:
        with open(f'pareto_adult_{metric}.pkl', 'rb') as f:
            cls_freqs = pickle.load(f)
    labels = list(cls_freqs.keys())
    width = 0.4
    eps = 0.6
    x_axis = [0]*len(labels)
    for i in range(len(labels)):
        x_axis[i] = i*(width*6)
        if i != 0:
            x_axis[i] += width*i
    x_axis = np.asarray(x_axis)
    avgList = None
    for index, dataset in enumerate(['synthetic', 'adult', 'bank', 'compas', 'credit']):
        if mode == 'labelBias':
            with open(f'pareto_labelBias_{dataset}_{metric}.pkl', 'rb') as f:
                cls_freqs = pickle.load(f)
        else:
            with open(f'pareto_{dataset}_{metric}.pkl', 'rb') as f:
                cls_freqs = pickle.load(f)
        y_data = []
        for i in labels:
            if i in cls_freqs:
                y_data.append(cls_freqs[i])
            else:
                y_data.append(0)
        if avgList is None:
            avgList = y_data
        else:
            for j,_ in enumerate(y_data):
                avgList[j] += y_data[j]
        plt.bar(x_axis + index*width, y_data, label=dataset, width=width, edgecolor='white')
        if index == 4:
            for j,_ in enumerate(avgList):
                avgList[j] /= 5
            plt.bar(x_axis + (index + 1)*width, avgList, label='average', width=width, edgecolor='white')
    
    # Draw vertical lines
    for i in x_axis:
        plt.axvline(x=i + (index+1)*width + width, color='gray')
    plt.xticks(x_axis + 2*width, labels, rotation=30, fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(fontsize=13)
    #plt.title('Pareto Optimality Frequency', fontsize=9)
    plt.xlim([-width,x_axis[-1]+6*width])
    plt.grid()
    plt.ylabel('Frequency', fontsize=15)
    if mode != 'labelBias':
        if lessBiasMode == False:
            plt.savefig(f'underRepresentation_pareto_{metric}.png', dpi=300, bbox_inches='tight')
        else:
            plt.savefig(f'underRepresentation_pareto_{metric}_lessBiasMode.png', dpi=300, bbox_inches='tight')
    else:
        if lessBiasMode == False:
            plt.savefig(f'labelBias_pareto_{metric}.png', dpi=300, bbox_inches='tight')
        else:
            plt.savefig(f'labelBias_pareto_{metric}_lessBiasMode.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
plotParetoBars(metric='eod', lessBiasMode=True)
#plotParetoBars(metric='dp')

In [ ]:
def labelBias_max_mean_plots(files, metric='eod', testSet='orig', dataset='synthetic', statType='mean', plotLimits=None, lessBiasMode=False):
    if dataset == 'compas':
        omitCls = ['meta_fair_fdr']
    else:
        omitCls = []
    axisTitles = {
        'dp': 'Statistical Parity Diff.',
        'eod': 'Equal Opportunity Difference'
    }
    markers = {
        'lr': '+',
        'jiang_nachum': 'o',
        'rew':'o',
        'fairgan':'o',
        'exp_grad_eodds':'^',
        'meta_fair_fdr':'^',
        'prej_remover':'^',
        'gerry_fair': '^',
        'cal_eq':'s',
        'eq':'s',
        'reject':'s'
    }
    colors = {
        'lr': '#1f77b4',
        'jiang_nachum': '#ff7f0e',
        'rew':'#2ca02c',
        'fairgan':'#d62728',
        'exp_grad_eodds':'#9467bd',
        'meta_fair_fdr':'#8c564b',
        'prej_remover':'#e377c2',
        'gerry_fair': '#7f7f7f',
        'cal_eq':'#bcbd22',
        'eq':'#17becf',
        'reject':'#1f77b4'
    }
    fig, ax = plt.subplots(ncols=2)
    fig.subplots_adjust(wspace=0.3)
    fig.set_figheight(3)
    fig.set_figwidth(6)
    overallStats = {}
    for file in files:
        if file == 'base':
            file='lr'
        if file not in omitCls:
            overallStats[file] = [[],[]]
    if lessBiasMode == True:
        end_range=5
    else:
        end_range = 10
    for lb,label_bias in enumerate(range(0,end_range)):
        label_bias /= 10
        # Calculating required Quantities
        for file in files:
            if file in omitCls:
                continue
            results = files[file]
            try:
                if metric == 'dp':
                    y_bal = abs(results['label_bias'][f'imbalance_{str(label_bias)}']['balanced'].statistical_parity_difference())
                    y_biased = abs(results['label_bias'][f'imbalance_{str(label_bias)}']['biased'].statistical_parity_difference())
                    y_orig = abs(results['label_bias'][f'imbalance_{str(label_bias)}']['original'].statistical_parity_difference())
                elif metric == 'eod':
                    y_bal = abs(results['label_bias'][f'imbalance_{str(label_bias)}']['balanced'].equal_opportunity_difference())
                    y_biased = abs(results['label_bias'][f'imbalance_{str(label_bias)}']['biased'].equal_opportunity_difference())
                    y_orig = abs(results['label_bias'][f'imbalance_{str(label_bias)}']['original'].equal_opportunity_difference())
                # Calculate accuracy
                x_err_rate_bal = (1 - abs(results['label_bias'][f'imbalance_{str(label_bias)}']['balanced'].accuracy()))
                x_err_rate_biased = (1 - abs(results['label_bias'][f'imbalance_{str(label_bias)}']['biased'].accuracy()))
                x_err_rate_orig = (1 - abs(results['label_bias'][f'imbalance_{str(label_bias)}']['original'].accuracy()))
            except TypeError as te:
                continue
            if testSet == 'bal':
                y = y_bal
                x = x_err_rate_bal
            elif testSet == 'biased':
                y = y_biased
                x = x_err_rate_biased
            elif testSet == 'orig':
                y = y_orig
                x = x_err_rate_orig
            if file == 'base':
                file='lr'
            overallStats[file][0].append(x)
            overallStats[file][1].append(y)
    ##############
    # Overall Mean
    ##############
    del overallStats['lr']
    for i in overallStats:
        if statType == 'max':
            ax[1].errorbar(x=max(overallStats[i][0]), y=max(overallStats[i][1]), label=i, fmt=markers[i], color=colors[i])
        else:
            if np.mean(overallStats[i][0]) > plotLimits[dataset][0][1] or np.mean(overallStats[i][1]) > plotLimits[dataset][1][1]:
                ax[1].errorbar(x=np.mean(overallStats[i][0]), y=np.mean(overallStats[i][1]), label=i, fmt=markers[i], color=colors[i])
            else:
                temp = ax[1].errorbar(x=np.mean(overallStats[i][0]), y=np.mean(overallStats[i][1]), xerr=np.std(overallStats[i][0]), yerr=np.std(overallStats[i][1]), label=i, fmt=markers[i], color=colors[i])
                [bar.set_alpha(0.3) for bar in temp[2]]
                [cap.set_alpha(0.3) for cap in temp[1]]
        ax[1].set_ylim(plotLimits[dataset][1])
        ax[1].set_xlim(plotLimits[dataset][0])
    # D2 distribution lines
    #if dataset != 'synthetic':
    if metric == 'eod':
        ax[1].axvline(x=(1.0 - files['base']['label_bias']['imbalance_0.0']['original'].accuracy()))
        ax[1].axhline(y=abs(files['base']['label_bias']['imbalance_0.0']['original'].equal_opportunity_difference()))
    elif metric == 'dp':
        ax[1].axvline(x=(1.0 - files['base']['label_bias']['imbalance_0.0']['original'].accuracy()))
        ax[1].axhline(y=abs(files['base']['label_bias']['imbalance_0.0']['original'].statistical_parity_difference()))
    ax[1].set_xlabel('Error Rate')
    #ax[1].set_ylabel(axisTitles[metric])
    ax[1].set_title('Mean Over all settings', fontsize=10)
#     else:
#         bayes_optimal_results = getBayesOptimalResults()
#         if metric == 'eod':
#             ax[1].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[1].axhline(y=abs(bayes_optimal_results.equal_opportunity_difference()))
#         elif metric == 'dp':
#             ax[1].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[1].axhline(y=abs(bayes_optimal_results.statistical_parity_difference()))
#         ax[1].set_xlabel('Error Rate')
#         ax[1].set_ylabel(axisTitles[metric])
#         ax[1].set_title('Mean Over all settings', fontsize=10)
    
    ################
    # Frequency Plot
    ################
    cls_freqs = {}
    for i in files:
        cls_freqs[i] = 0
    del cls_freqs['base']
    for label_bias in range(0,end_range):
        label_bias /= 10
        current_coord = {}
        for file in files:
            try:
                if metric == 'eod':
                    current_coord[file] = (1 - files[file]['label_bias'][f'imbalance_{label_bias}']['original'].accuracy(), abs(files[file]['label_bias'][f'imbalance_{label_bias}']['original'].equal_opportunity_difference()))
                elif metric == 'dp':
                    current_coord[file] = (1 - files[file]['label_bias'][f'imbalance_{label_bias}']['original'].accuracy(), abs(files[file]['label_bias'][f'imbalance_{label_bias}']['original'].statistical_parity_difference()))
            except TypeError as te:
                continue
        del current_coord['base']
        sorted_acc = sorted(list(current_coord.items()), key=lambda x: x[1][0])
        sorted_fairness = sorted(list(current_coord.items()), key=lambda x: x[1][1])
        k = 5
        top_k_classifiers = list(set([i[0] for i in sorted_acc][:k]).intersection(set([i[0] for i in sorted_fairness][:k])))
        for i in current_coord:
            if current_coord[i][0] < 0.5 and current_coord[i][1] < 0.5:
                flag = 0
                for j in current_coord:
                    if j != i:
                        # Check whether a point exists
                        if current_coord[j][0] < current_coord[i][0] and current_coord[j][1] < current_coord[i][1]:
                            flag = 1
                            break
                if flag == 0 and i in top_k_classifiers:
                    cls_freqs[i] += 1

    ###############################
    # Original Split Results
    ###############################
    for i in files:
        if i not in omitCls:
            x = 1 - files[i]['label_bias'][f'imbalance_0.0']['original'].accuracy()
            if metric == 'eod':
                y=abs(files[i]['label_bias'][f'imbalance_0.0']['original'].equal_opportunity_difference())
            elif metric == 'dp':
                y=abs(files[i]['label_bias'][f'imbalance_0.0']['original'].statistical_parity_difference())
            if i == 'base':
                i = 'lr'
            ax[0].errorbar(x,y, label=i, fmt=markers[i], color=colors[i])
    # D2 distribution lines
    #if dataset != 'synthetic':
    if metric == 'eod':
        ax[0].axvline(x=(1.0 - files['base']['label_bias']['imbalance_0.0']['original'].accuracy()))
        ax[0].axhline(y=abs(files['base']['label_bias']['imbalance_0.0']['original'].equal_opportunity_difference()))
    elif metric == 'dp':
        ax[0].axvline(x=(1.0 - files['base']['label_bias']['imbalance_0.0']['original'].accuracy()))
        ax[0].axhline(y=abs(files['base']['label_bias']['imbalance_0.0']['original'].statistical_parity_difference()))
#     else:
#         bayes_optimal_results = getBayesOptimalResults()
#         if metric == 'eod':
#             ax[0].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[0].axhline(y=abs(bayes_optimal_results.equal_opportunity_difference()))
#         elif metric == 'dp':
#             ax[0].axvline(x=(1.0 - bayes_optimal_results.accuracy()))
#             ax[0].axhline(y=abs(bayes_optimal_results.statistical_parity_difference()))
    ax[0].set_xlabel('Error Rate')
    ax[0].set_ylabel(axisTitles[metric])
    ax[0].set_title('Results on Original Split', fontsize=10)
    ax[0].set_ylim(plotLimits[dataset][1])
    ax[0].set_xlim(plotLimits[dataset][0])
    
    if dataset == 'credit':
        plt.legend(bbox_to_anchor=(1.1,1.1))
    elif dataset == 'adult':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    elif dataset == 'compas':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    elif dataset == 'bank':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    elif dataset == 'synthetic':
        plt.legend(loc='right', bbox_to_anchor=(2.1,0.5))
    plt.show()
    
    if lessBiasMode == True:
        fig.savefig(f'{dataset}_{metric}_lessBias_labelBias.png', dpi=300, bbox_inches='tight')
    else:
        fig.savefig(f'{dataset}_{metric}_labelBias.png', dpi=300, bbox_inches='tight')
    
    # Dump Pareto Plot data
    with open(f'pareto_labelBias_{dataset}_{metric}.pkl', 'wb') as f:
        pickle.dump(cls_freqs, f)
    plt.figure()
    plt.bar(range(len(cls_freqs)), cls_freqs.values(), tick_label=list(cls_freqs.keys()))
    plt.xticks(rotation=90, fontsize=8)
    plt.title('Pareto Optimality Frequency', fontsize=9)
    plt.show()

In [ ]:
metric = 'eod'
if metric == 'eod':
    plotLimits = {
        'synthetic': ([0, 0.2], [-0.05, 0.4]),
        'adult': ([0, 0.35], [-0.01, 0.2]),
        'bank': ([0, 0.15], [-0.01, 0.1]),
        'compas': ([0.05, 0.55], [-0.01, 0.2]),
        'credit': ([0, 0.4], [-0.01, 0.3])
    }
else:
    plotLimits = {
            'synthetic': ([-0.05, 0.25], [0.15,0.4]),
            'adult': ([-0.1, 0.4], [-0.01, 0.3]),
            'bank': ([0, 0.15], [-0.01, 0.1]),
            'compas': ([0.05, 0.5], [-0.01, 0.3]),
            'credit': ([0, 0.4], [-0.02, 0.3])
        }
labelBias_max_mean_plots(files, metric=metric, dataset=dataset, plotLimits=plotLimits, lessBiasMode=True)
#labelBias_max_mean_plots(files, metric=metric, dataset=dataset, plotLimits=plotLimits)

In [ ]:
plotParetoBars(mode='labelBias', metric='eod', lessBiasMode=True)
#plotParetoBars(mode='labelBias', metric='dp')